<a href="https://colab.research.google.com/github/Glifoyle/test-repo/blob/master/Copy_of_6_w2v_v7_gensim_livedoor_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 目的：単語ベクトルと日本語の自然言語処理を理解する

## テーマ1：単語ベクトルと，単語ベクトルによる単語間の演算を理解する
## テーマ2：発展課題：日本語自然言語処理を体験する（単語ベクトルを取得する）

<br>
&copy; 2021-2024 Hiro Kimura
<br>


## ポイント

この教材では以下を行う：
1. 学習済み単語ベクトルで演算を行う
  1. ベクトル値
  1. 単語間の距離
  1. 類似度


2. 学習により単語ベクトルを生成する（発展課題）
  1. Livedoorニュースデータからコーパスを作成する
    1. ダウンロード
    1. データの結合
    1. データとラベルの整理
  1. 日本語解析用ツールをインストール
    1. gensim
    1. MeCab + ipadic
  1. コーパスからデータセットを作成する
    1. 形態素解析（単語に分割）
    1. クリーニング （不要なタグ記号等を除く）
    1. 正規化（表記の統一など）
    1. 単語の数値化
  1. 単語間の関係を考慮した単語ベクトルの生成
    1. データの前処理
    1. 学習用データセットの作成
    1. Word2Vecの学習
  1. 単語ベクトルで様々な演算を行う
    1. 類似度・距離
    1. 加減算  


# 実習の準備をする
  1. GPUを確認する
  1. タイムゾーンを東京にする
  1. Google Driveをマウントする
  1. 作業フォルダを作成する

<!--
  1. ライブラリを導入する
  1. 開発環境を整備する
-->




## GPUを確認する

In [ ]:
import torch

# GPUの設定であればcuda，そうでなければcpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# 割り当てられたGPUを調べる
!nvidia-smi

Wed Jul 10 04:02:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8              12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## タイムゾーンを東京に変更する

In [ ]:
!date

Wed Jul 10 04:02:47 AM UTC 2024


In [ ]:
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Tokyo /etc/localtime

In [ ]:
!date

Wed Jul 10 01:02:54 PM JST 2024


## Google Driveをマウントする

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 作業フォルダを作成する

In [ ]:
# 作業フォルダ（作業ディレクトリ）を作成
import os

chap_dir = "/content/drive/MyDrive/practice/chap_w2v"
if not os.path.exists(chap_dir): # ディレクトリが存在していなければ
    # os.mkdir(chap_dir)
    os.makedirs(chap_dir) # ディレクトリを，途中のディレクトリも含めて，作成する

os.chdir(chap_dir) # 今回の作業ディレクトリに移動する
os.getcwd() # 現在の作業ディレクトリを表示する

'/content/drive/MyDrive/practice/chap_w2v'

# テーマ1：単語ベクトルで演算を行う
大規模な日本語コーパスとして日本語Wikipediaのデータで日本語学習済みの単語ベクトルを用いて，単語間の演算を行い，単語分散表現の効果を確認する
1. 東北大学が公開しているエンティティベクトル（学習済み単語ベクトル）をダウンロードして解凍する
2. エンティティベクトルをWord2Vecのフォーマットに変換する
3. 単語間の演算を行う

## 単語ベクトル（Word2Vec）の導入

In [ ]:
# # 現在のディレクトリを確認
! pwd
! ls -F

/content/drive/MyDrive/practice/chap_w2v


In [ ]:
# エンティティベクトルをダウンロード（1.3GB，約2分かかる）
! wget "http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2"

--2024-07-10 13:11:57--  http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2
Resolving www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1373795477 (1.3G) [application/x-bzip2]
Saving to: ‘20170201.tar.bz2’

20170201.tar.bz2    100%[===================>]   1.28G  21.9MB/s    in 61s     

2024-07-10 13:12:58 (21.6 MB/s) - ‘20170201.tar.bz2’ saved [1373795477/1373795477]



In [ ]:
# ファイルを確認
! ls -F

20170201.tar.bz2


In [ ]:
# ファイルを解凍する（3分30秒程度かかる）
# ./jawiki/ 以下のフォルダが作成される

import tarfile
import os

# 解凍
tar = tarfile.open("./20170201.tar.bz2", "r:bz2")
tar.extractall("./jawiki/")
tar.close()


In [ ]:
# ファイルを確認
! ls -F

20170201.tar.bz2  jawiki/


# 単語間の演算を行う

- gensin：単語ベクトルを扱うツール


In [ ]:
# エンティティベクトルをw2vのフォーマットに変換する（約30秒かかる）

from gensim.models import KeyedVectors

model_jw = KeyedVectors.load_word2vec_format(
    "./jawiki/entity_vector/entity_vector.model.bin", binary=True)


In [ ]:
# ファイルに保存する（約3分かかる）
model_jw.save_word2vec_format("./jawiki/jawiki_w2v.vec")

In [ ]:
# 単語ベクトル（200次元）を確認する
wv1 = model_jw['山形']
print(wv1.shape)
print(wv1)

(200,)
[ 1.0997447   0.8196625   0.8369182  -1.2915025  -0.06917344  0.16544713
 -1.2095165  -0.9141803  -0.18874143  0.61299783 -1.1284201   1.127517
 -0.40697736 -0.3943502   3.0227787  -0.27428445  0.34976038  0.892071
  0.6544367  -0.09852517 -0.27865645 -1.2212168  -0.5908141  -0.909622
  0.811184   -0.23257841 -0.13242802 -1.3306999  -0.42543045 -1.5823678
 -0.06474027 -0.6807919   0.93179566 -1.4261703   1.5826771   0.23798552
  0.15092313  1.0781382   0.97536963  1.0345681  -0.4893855   1.2355504
 -1.1773647  -0.5797864   0.87899554  0.31232962 -0.07045349 -0.1570589
  0.86451596  1.875505    1.1548423  -1.2485168   1.5853759  -1.4500644
  0.90929335  1.2907407  -0.26110485 -0.23115727 -0.22397086 -0.41572917
 -0.35956278  0.96473366 -1.7440711  -0.875481   -0.04452347  1.6692209
 -0.839226    0.33410925  1.1115055   0.21318859 -0.46992478  0.03419644
 -0.63869476  0.9341197   0.13374513  0.10631881  0.7268205  -0.4531535
 -1.2031064  -0.65960604  0.8011626   2.368447    0.2688

類似度（コサイン類似度）：
\begin{align}
コサイン類似度 & = \frac{Vec1とVec2の内積}{(Vec1の大きさ)*(Vec2の大きさ)}
  & = \frac{a_1b_1+a_2b_2+...+a_nb_n}{\sqrt{a_1^2+a_2^2+...+a_n^2} \sqrt{b_1^2+b_2^2+...+b_n^2}}
\end{align}

In [ ]:
# 単語の類似度
similarity = model_jw.similarity('山形', '山形')
print(similarity)

1.0


In [ ]:
# 単語の類似度
similarity = model_jw.similarity('山形', 'ニューヨーク')
# distance = model_jw.distance('山形', '南極')
print(similarity)

0.085347414


In [ ]:
# 単語の距離
distance = model_jw.distance('山形', '山形')
print(distance)

0.0


In [ ]:
# 単語の距離
distance = model_jw.distance('山形', 'ニューヨーク')
# distance = model_jw.distance('山形', '南極')
print(distance)

0.9146525859832764


In [ ]:
# 単語の演算　山形と近いのは
words = model_jw.similar_by_word('山形', topn=10)
for word in words:
  print(word)

('秋田', 0.811007022857666)
('福島', 0.7636185884475708)
('栃木', 0.7335861325263977)
('香川', 0.7301021814346313)
('長野', 0.7231136560440063)
('岩手', 0.7200448513031006)
('大分', 0.7196505665779114)
('青森', 0.719536304473877)
('仙台', 0.7151240110397339)
('熊本', 0.7114702463150024)


In [ ]:
# 単語の演算　山形と川と言ったら？
words = model_jw.most_similar(positive=['山形','河川'], topn=5)
for word in words:
  print(word)

('[最上川]', 0.7070518136024475)
('[旭川_(岡山県)]', 0.6735597252845764)
('[信濃川]', 0.6719672679901123)
('[阿賀野川]', 0.6703377366065979)
('本川', 0.6636521816253662)


In [ ]:
# 類似の単語
words = model_jw.most_similar(positive=['フルート', 'ピアノ'], topn=3)
for word in words:
  print(word)

('チェロ', 0.9055847525596619)
('[ピアノ]', 0.8905407190322876)
('ヴァイオリン', 0.8903535008430481)


In [ ]:
# 単語の演算　王様 - 男 + 女　は？（cos距離）
words = model_jw.most_similar_cosmul(positive=['王様', '女'], negative=['男'], topn=1)
for word in words:
  print(word)

('お姫様', 0.9508086442947388)


In [ ]:
# 単語の演算　王様 - 男 + 女　は？
words = model_jw.most_similar(positive=['王様','女'], negative=['男'], topn=1)
for word in words:
  print(word)

('お姫様', 0.7259090542793274)


In [ ]:
# 単語の演算　ロンドン - イギリス + 日本　は？
words = model_jw.most_similar(positive=['ロンドン','日本'], negative=['イギリス'], topn=1)
for word in words:
  print(word)

('東京', 0.6461172103881836)


In [ ]:
# 仲間外れの単語は？

l = ["金沢", "ピアノ", "盛岡", "仙台", "水戸", "甲府"]
words = model_jw.doesnt_match(l)
print(words)

ピアノ


In [ ]:
# 仲間外れの単語は？

l = ["金沢", "福岡", "盛岡", "仙台", "水戸", "甲府"]
words = model_jw.doesnt_match(l)
print(words)

福岡


In [ ]:
# 単語の演算　猫といったら？
words = model_jw.similar_by_word('猫', topn=10)
for word in words:
  print(word)

('[ネコ]', 0.8454327583312988)
('ネコ', 0.8190229535102844)
('[猫]', 0.8119831681251526)
('ウサギ', 0.8038156032562256)
('小鳥', 0.7744659781455994)
('子猫', 0.7610605955123901)
('[犬]', 0.7583129405975342)
('ネズミ', 0.7523773908615112)
('子犬', 0.7486448884010315)
('犬', 0.7439809441566467)


In [ ]:
# 単語の演算　猫といったら？
words = model_jw.similar_by_word('ねこ', topn=10)
for word in words:
  print(word)

('いぬ', 0.7891684174537659)
('うさぎ', 0.7774231433868408)
('みみ', 0.7745429277420044)
('くじら', 0.7738558053970337)
('とー', 0.773608386516571)
('にゃん', 0.7709331512451172)
('ひな', 0.7699280381202698)
('ひみつ', 0.7685192227363586)
('えほん', 0.761918306350708)
('なぞ', 0.7601579427719116)


# テーマ2：発展課題：学習により単語ベクトルを取得する

  1. Livedoorニュースデータからコーパスを作成する
    1. ダウンロード
    1. クリーニング （不要なタグ記号等を除く）
    1. データの結合
    1. データとラベルの整理
  1. 日本語解析用ツールをインストール
    1. gensim
    1. MeCab + ipadic
  1. コーパスからデータセットを作成する
    1. 形態素解析（単語に分割）
    1. クリーニング （不要なタグ記号等を除く）
    1. 正規化（表記の統一など）
    1. 単語の数値化
  1. 単語間の関係を考慮した単語ベクトルの生成
    1. データの前処理
    1. 学習用データセットの作成
    1. Word2Vecの学習
  1. 単語ベクトルで様々な演算を行う
    1. 類似度・距離
    1. 加減算  

# コーパスを作成する

## Livedoorニュースデータ（2014年2月9日）を取得する
1. [ldcc-20140209.tar.gz](https://www.rondhuit.com/download.html#ldcc) を Google Driveの '/content/drive/MyDrive/practice/' の実習用のカレントディレクトリにダウンロードする


2. ダウンロードした ZIPファイルを， 実習用のカレントディレクトリに解凍する


In [ ]:
# # 現在のディレクトリを確認
!pwd
!ls -F

/content/drive/MyDrive/practice/chap_w2v
20170201.tar.bz2  jawiki/


In [ ]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
# !curl -OL "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2024-07-10 13:37:15--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  2.63MB/s    in 3.2s    

2024-07-10 13:37:19 (2.63 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [ ]:
!ls -F

20170201.tar.bz2  jawiki/  ldcc-20140209.tar.gz


In [ ]:
# dest_pathが存在しないとき
# tarファイルを展開する（70秒程度）

import os

dest_path = "./dataset/livedoor/"     # 解凍先ディレクトリ
tar_path = "./ldcc-20140209.tar.gz"  # 解凍するtarファイル

# ファイルを展開する（70秒程度）
if not os.path.exists(dest_path):
    tar = tarfile.open(tar_path, "r:gz")
    tar.extractall(dest_path)
    tar.close()
else:
    print(f"{dest_path} already exists!")

In [ ]:
# カレントディレクトリ以下のファイルとディレクトリを確認
# ./dataset/livedoor/text/ にカテゴリー別にフォルダで分離されて記事が配置されている
# フォルダ名がカテゴリー名，フォルダ数がカテゴリー数

# フォルダ名（=カテゴリー名）をリストとして表示する
files_folders = [name for name in os.listdir("./dataset/livedoor/text/")]
print(files_folders)
print(len(files_folders))

['CHANGES.txt', 'dokujo-tsushin', 'it-life-hack', 'kaden-channel', 'livedoor-homme', 'movie-enter', 'peachy', 'README.txt', 'smax', 'sports-watch', 'topic-news']
11


In [ ]:
# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./dataset/livedoor/text/") if os.path.isdir("./dataset/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)

カテゴリー数: 9
['dokujo-tsushin', 'it-life-hack', 'kaden-channel', 'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch', 'topic-news']


In [ ]:
# 記事ファイルの構造を確認してみる
# 一例として、movie-enterカテゴリーの一つの記事
file_name = "./dataset/livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name, encoding='utf-8') as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文
    print("-1：", text[-1])  # 本文
    print("n：", len(text))  # 本文

0： http://news.livedoor.com/article/detail/6255260/

1： 2012-02-07T09:00:00+0900

2： 新しいヴァンパイアが誕生！　ジョニデ主演『ダーク・シャドウ』の公開日が決定

3： 　こんなヴァンパイアは見たことがない！　ジョニー・デップとティム・バートン監督がタッグを組んだ映画『ダーク・シャドウズ（原題）』の邦題が『ダーク・シャドウ』に決定。日本公開日が5月19日に決まった。さらに、ジョニー・デップ演じるヴァンパイアの写真が公開された。

-1： ・ティム・バートンとジョニデが8度目のタッグ、念願のバンパイア役を演じる

n： 17


## クリーニングとデータの抽出



In [ ]:
# 本文を取得する前処理（クリーニング）関数を定義

def extract_main_txt(file_name):
    with open(file_name, encoding='utf-8') as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 具体的なクリーニング内容
        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text


In [ ]:
# 前処理した本文と、カテゴリーのラベルをリストに追加していく
# 約1分（負荷状況により10秒～20分程度の場合もある）
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./dataset/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施しながら本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)


In [ ]:
# 例として，1番目の文章とラベルを確認
# 本文の総数は 7,376

print("最初の本文：",list_text[0])
print("最初のラベル：", list_label[0])
print("本文の総数：", len(list_text))

最初の本文： もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり……という「お祝い貧乏」状態の人も多いのではないだろうか？さらに出席回数を重ねていくと、こんなお願いごとをされることも少なくない。「お願いがあるんだけど……友人代表のスピーチ、やってくれないかな？」さてそんなとき、独女はどう対応したらいいか？最近だとインターネット等で検索すれば友人代表スピーチ用の例文サイトがたくさん出てくるので、それらを参考にすれば、無難なものは誰でも作成できる。しかし由利さん（33歳）はネットを参考にして作成したものの「これで本当にいいのか不安でした。一人暮らしなので聞かせて感想をいってくれる人もいないし、かといって他の友人にわざわざ聞かせるのもどうかと思うし……」ということで活用したのが、なんとインターネットの悩み相談サイトに。そこに作成したスピーチ文を掲載し「これで大丈夫か添削してください」とメッセージを送ったというのである。「一晩で3人位の人が添削してくれましたよ。ちなみに自分以外にもそういう人はたくさんいて、その相談サイトには同じように添削をお願いする投稿がいっぱいありました」（由利さん）。ためしに教えてもらったそのサイトをみてみると、確かに「結婚式のスピーチの添削お願いします」という投稿が1000件を超えるくらいあった。めでたい結婚式の影でこんなネットコミュニティがあったとは知らなかった。しかし「事前にお願いされるスピーチなら準備ができるしまだいいですよ。一番嫌なのは何といってもサプライズスピーチ！」と語るのは昨年だけで10万以上お祝いにかかったというお祝い貧乏独女の薫さん（35歳）「私は基本的に人前で話すのが苦手なんですよ。だからいきなり指名されるとしどろもどろになって何もいえなくなる。そうすると自己嫌悪に陥って終わった後でもまったく楽しめなくなりますね」サプライズスピーチのメリットとしては、準備していない状態なので、フランクな本音をしゃべってもらえるという楽しさがあるようだ。しかしそれも上手に対応できる人ならいいが、苦手な人の場合だと「フランク」ではなく「しどろもどろ」になる危険性大。ちなみにプロの司会者の場合、本当のサプライズではなく式の最中に「のちほどサプライズスピーチとしてご指名させていただきます」という一言があることも多いよ

In [ ]:
# リストデータをpandasのDataFrameに読み込む
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

# print(df.head())
df.head()


(7376, 2)


,text,label
0,もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり……...,dokujo-tsushin
1,携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通だった。恋人と別れたら、手帳に書か...,dokujo-tsushin
2,「男性はやっぱり、女性の“すっぴん”が大好きなんですかね」と不満そうに話すのは、出版関係で働...,dokujo-tsushin
3,ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは「そげる→たわむ→外に流れる...,dokujo-tsushin
4,6月から支給される子ども手当だが、当初は子ども一人当たり月額2万6000円が支給されるはずだ...,dokujo-tsushin


In [ ]:
# カテゴリーの辞書を作成
# 双方向：番号 ⇒ カテゴリー名，カテゴリー名 ⇒ 番号

dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

print(dic_id2cat)
print(dic_cat2id)


{0: 'dokujo-tsushin', 1: 'it-life-hack', 2: 'kaden-channel', 3: 'livedoor-homme', 4: 'movie-enter', 5: 'peachy', 6: 'smax', 7: 'sports-watch', 8: 'topic-news'}
{'dokujo-tsushin': 0, 'it-life-hack': 1, 'kaden-channel': 2, 'livedoor-homme': 3, 'movie-enter': 4, 'peachy': 5, 'smax': 6, 'sports-watch': 7, 'topic-news': 8}


In [ ]:
# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)
df.head()

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]
df.head()

,text,label_index
0,もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり……...,0
1,携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通だった。恋人と別れたら、手帳に書か...,0
2,「男性はやっぱり、女性の“すっぴん”が大好きなんですかね」と不満そうに話すのは、出版関係で働...,0
3,ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは「そげる→たわむ→外に流れる...,0
4,6月から支給される子ども手当だが、当初は子ども一人当たり月額2万6000円が支給されるはずだ...,0


In [ ]:
print(df.shape)

(7376, 2)


In [ ]:
# ここまでのデータ（本文とラベル番号）をcsvファイルに保存する

import pickle

# データフレームをファイルに保存する
out_path = "./dataset/livedoor/all_seq.csv"
df.to_csv(out_path, index=False)

# 辞書をファイルに保存する
with open("./dataset/livedoor/dic_id2cat.pkl", mode='wb') as f1:
  pickle.dump(dic_id2cat, f1)

with open ("./dataset/livedoor/dic_cat2id.pkl", mode='wb') as f2:
  pickle.dump(dic_cat2id, f2)

In [ ]:
!pwd
!ls -l
!ls -l dataset
!ls -l dataset/livedoor

/content/drive/MyDrive/practice/chap_w2v
total 1350254
-rw------- 1 root root 1373795477 Feb 18  2017 20170201.tar.bz2
drwx------ 3 root root       4096 Jul 10 13:38 dataset
drwx------ 3 root root       4096 Jul 10 13:27 jawiki
-rw------- 1 root root    8855190 Feb  9  2014 ldcc-20140209.tar.gz
total 4
drwx------ 3 root root 4096 Jul 10 13:51 livedoor
total 23752
-rw-------  1 root root 24316399 Jul 10 13:51 all_seq.csv
-rw-------  1 root root      157 Jul 10 13:51 dic_cat2id.pkl
-rw-------  1 root root      157 Jul 10 13:51 dic_id2cat.pkl
drwx------ 11 root root     4096 Feb  9  2014 text


# 日本語処理用のツールを導入する
- 形態素解析ライブラリである**MeCab**と**ipadic**を使用する
- Pythonからの利用では、さらに**mecab-python**を使用する

In [ ]:
# MeCabとtransformersの準備（約30秒～50秒）
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
# !pip install transformers==2.9.0 # これは動作する
# # !pip install transformers==3.0.2    # tokenizer等作れず

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30 swig4.0
Suggested packages:
  apt-xapian-index aptitude-doc-en | aptitude-doc debtags tasksel libcwidget-dev xapian-tools
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30 swig swig4.0
0 upgraded, 7 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,954 kB of archives.
After this operation, 22.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aptitude-common all 0.8.13-3ubuntu1 [1,719 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsigc++-2.0-0v5 amd64 2.10.4-2ubuntu3 [12.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcwidget4 amd64 0.5.18-5build1 [306 kB]
Get:

In [ ]:
# シンボリックリンクを設定（念のためエラー対応）
# !find / -iname mecabrc

!ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [ ]:
!ls -l /usr/local/etc/

total 4
drwxr-xr-x 1 root root 4096 Jul  9 05:06 jupyter
lrwxrwxrwx 1 root root   12 Jul 10 13:52 mecabrc -> /etc/mecabrc


## 日本語の分かち書き（形態素解析・単語分割）の確認

In [ ]:
# Mecabの動作確認
import MeCab

# 分かち書きだけ出力
m=MeCab.Tagger("-Owakati")

text = "私は深層学習の勉強を始めました。"

text_segmented = m.parse(text)
print(text_segmented)

私 は 深層 学習 の 勉強 を 始め まし た 。 



In [ ]:
# 品詞の表示

m=MeCab.Tagger("-Ochasen")

text = "私は深層学習の勉強を始めました。"

text_segmented = m.parse(text)
print(text_segmented)

私	ワタシ	私	名詞-代名詞-一般		
は	ハ	は	助詞-係助詞		
深層	シンソウ	深層	名詞-一般		
学習	ガクシュウ	学習	名詞-サ変接続		
の	ノ	の	助詞-連体化		
勉強	ベンキョウ	勉強	名詞-サ変接続		
を	ヲ	を	助詞-格助詞-一般		
始め	ハジメ	始める	動詞-自立	一段	連用形
まし	マシ	ます	助動詞	特殊・マス	連用形
た	タ	た	助動詞	特殊・タ	基本形
。	。	。	記号-句点		
EOS



In [ ]:
# ヨミ（読み）の出力
m=MeCab.Tagger("-Oyomi")

text = "私は深層学習の勉強を始めました。"

text_segmented = m.parse(text)
print(text_segmented)

ワタシハシンソウガクシュウノベンキョウヲハジメマシタ。



# コーパスからデータセットを作成する
1. コーパスの作成
  - テキストデータを連結
1. 前処理
  1. クリーニングを行う
    - 不要な文字を削除
  1. 正規化処理を行う
    - 文字コードを統一
    - アルファベットを小文字に統一
  1. 分かち書きにする
1. データセットの作成

In [ ]:
# データフレームの列名を確認する
print(df.columns)

Index(['text', 'label_index'], dtype='object')


In [ ]:
# データフレームのデータ構造を確認する
print(df.shape)
# print(df['text'].head(10))
print(df.head(10))

(7376, 2)
                                                text  label_index
0  もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり……...            0
1  携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通だった。恋人と別れたら、手帳に書か...            0
2  「男性はやっぱり、女性の“すっぴん”が大好きなんですかね」と不満そうに話すのは、出版関係で働...            0
3  ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは「そげる→たわむ→外に流れる...            0
4  6月から支給される子ども手当だが、当初は子ども一人当たり月額2万6000円が支給されるはずだ...            0
5  書店で偶然『うさぎのくれたバレエシューズ』(小峰書店 安房直子/著、南塚直子/イラスト)とい...            0
6  昨年の秋、希望の職種に転職したカナコさん（30歳／ 商社勤務）。やりがいのある仕事を得て充実...            0
7  「彼の収入が少ないから私も働かなければならないし、それを思うと結婚はもう少し先でもいいかな」...            0
8  これからの季節、お肌の天敵と言えば“紫外線”。マーケティング会社トレンダーズ株式会社が、20...            0
9  合コンやパーティー、そして料理教室から農業まで。さまざまなスタイルで男女の人気を集める婚カツ...            0


In [ ]:
# クリーニング処理を定義する
def text_cleaning(df):
  # 不要な文字を消す
  stop_chars = "\n,.、。・, 、()（）「」「　『 』[]【】“”!！ ?？—:：→・■●★▲▼"
  for stop_char in stop_chars:
      df["text"] = df["text"].str.replace(stop_char, " ") # stop_char をブランクに

  return df

In [ ]:
# 正規化処理を定義する
def text_norm(df):
  # ユニコード正規化
  df["text"] = df["text"].str.normalize("NFKC")

  # アルファベットを小文字に統一
  df["text"] = df["text"].str.lower()

  return df

In [ ]:
# tokenizerを定義する

import MeCab

tagger = MeCab.Tagger("-Owakati")

def mecab_tokenizer(text):
    # 半角スペースで区切られた単語列
    tokens = tagger.parse(text)

    # 半角スペースで区切ったリスト
    # tokens_list = tokens.sprip().split()
    tokens_list = tokens.split()


    # テキストを分かち書きする関数を準備する
    # parsed_lines = tagger.parse(text).split("\n")[:-2]
    # surfaces = [l.split('\t')[0] for l in parsed_lines]
    # features = [l.split('\t')[1] for l in parsed_lines]
    # # 原型を取得
    # bases = [f.split(',')[6] for f in features]
    # # 配列で結果を返す
    # token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # # アルファベットを小文字に統一
    # token_list = [t.lower() for t in token_list]
    return tokens_list


In [ ]:
# クリーニング処理の確認
df_1 = text_cleaning(df)

# print(df['text'].head(10))
# print()
# print(df_1['text'].head(10))
print(df['text'][100:110])
print()
print(df_1['text'][100:110])
# print(df['text'].tail())
# print()
# print(df_1['text'].tail())

100    夏の疲れもあってか肌荒れに悩まされているミサトさん 34歳 営業  ファンデーションのノリも...
101     ミニブログ と呼ばれるツイッター人気については 今さら説明する必要もないだろう 140文字...
102    歳を経ていくと 世の中の仕組みや現実というものが イヤと言うほどわかってくるというもの しか...
103    独女なら これまで1度や2度はこんなことを言われた経験をお持ちではないだろうか 霧子さん 3...
104    美人は生まれたときから得をすることが多いように思える 何がいちばん得かって それはモテること...
105    十数年前 初めて100円均一ショップ 以下100均 に行ったときは驚いた 食品から日用品 ち...
106    この夏 地方都市に住む和歌子さん 28歳 は 母親の友人から見合いを勧められた 相手は35歳...
107    先日 同窓会の幹事グループに加わったサキさん 独身／41歳 会社員  仲が良かった男友達 K...
108    草食系男子 号泣男子 スカート男子 乙男 オトメン  女装子 ジョソコ など 化粧をしたり ...
109    先日 ある企業が主催する座談会で 元気な女性グループと知り合った  ゆりちゃん  まこちゃん...
Name: text, dtype: object

100    夏の疲れもあってか肌荒れに悩まされているミサトさん 34歳 営業  ファンデーションのノリも...
101     ミニブログ と呼ばれるツイッター人気については 今さら説明する必要もないだろう 140文字...
102    歳を経ていくと 世の中の仕組みや現実というものが イヤと言うほどわかってくるというもの しか...
103    独女なら これまで1度や2度はこんなことを言われた経験をお持ちではないだろうか 霧子さん 3...
104    美人は生まれたときから得をすることが多いように思える 何がいちばん得かって それはモテること...
105    十数年前 初めて100円均一ショップ 以下100均 に行ったときは驚いた 食品から日用品 ち...
106    この夏 地方都市に住む和歌子さん 28歳 は 母親の友人から見合いを勧められた 相手は35歳...
107 

In [ ]:
# 正規化処理の確認
df_2 = text_norm(df_1)

print(df_1['text'][100:110])
print()
print(df_2['text'][100:110])

100    夏の疲れもあってか肌荒れに悩まされているミサトさん 34歳 営業  ファンデーションのノリも...
101     ミニブログ と呼ばれるツイッター人気については 今さら説明する必要もないだろう 140文字...
102    歳を経ていくと 世の中の仕組みや現実というものが イヤと言うほどわかってくるというもの しか...
103    独女なら これまで1度や2度はこんなことを言われた経験をお持ちではないだろうか 霧子さん 3...
104    美人は生まれたときから得をすることが多いように思える 何がいちばん得かって それはモテること...
105    十数年前 初めて100円均一ショップ 以下100均 に行ったときは驚いた 食品から日用品 ち...
106    この夏 地方都市に住む和歌子さん 28歳 は 母親の友人から見合いを勧められた 相手は35歳...
107    先日 同窓会の幹事グループに加わったサキさん 独身/41歳 会社員  仲が良かった男友達 k...
108    草食系男子 号泣男子 スカート男子 乙男 オトメン  女装子 ジョソコ など 化粧をしたり ...
109    先日 ある企業が主催する座談会で 元気な女性グループと知り合った  ゆりちゃん  まこちゃん...
Name: text, dtype: object

100    夏の疲れもあってか肌荒れに悩まされているミサトさん 34歳 営業  ファンデーションのノリも...
101     ミニブログ と呼ばれるツイッター人気については 今さら説明する必要もないだろう 140文字...
102    歳を経ていくと 世の中の仕組みや現実というものが イヤと言うほどわかってくるというもの しか...
103    独女なら これまで1度や2度はこんなことを言われた経験をお持ちではないだろうか 霧子さん 3...
104    美人は生まれたときから得をすることが多いように思える 何がいちばん得かって それはモテること...
105    十数年前 初めて100円均一ショップ 以下100均 に行ったときは驚いた 食品から日用品 ち...
106    この夏 地方都市に住む和歌子さん 28歳 は 母親の友人から見合いを勧められた 相手は35歳...
107 

In [ ]:
# 分かち書きしたデータセットを作成
sentences = df_2["text"].apply(mecab_tokenizer)

# 作成されたデータのサンプル
print(len(sentences))
print(sentences[100:105])


7376
100    [夏, の, 疲れ, も, あっ, て, か, 肌荒れ, に, 悩まさ, れ, て, いる,...
101    [ミニブログ, と, 呼ば, れる, ツイッター, 人気, について, は, 今さら, 説明...
102    [歳, を, 経, て, いく, と, 世の中, の, 仕組み, や, 現実, という, も...
103    [独, 女, なら, これ, まで, 1, 度, や, 2, 度, は, こんな, こと, ...
104    [美人, は, 生まれ, た, とき, から, 得, を, する, こと, が, 多い, よ...
Name: text, dtype: object


# 学習により単語ベクトルを取得する

- データセット
  - 単語リスト
  - クリーニング処理，正規化処理、分かち書き処理済み
- Word2Vecで単語間の関係を学習
  - ベクトルの次元数：200
  - 最低出現頻度：5
  - ウィンドゥ幅：15
  - 学習アルゴリズム：skip-gram

In [ ]:
# Word2Veb用のディレクトリを作成する
w2v_dir = "./w2v"

if not os.path.exists(w2v_dir):
    os.mkdir(w2v_dir)

os.getcwd()
print(os.listdir())

['20170201.tar.bz2', 'jawiki', 'ldcc-20140209.tar.gz', 'dataset', 'w2v']


In [ ]:
!ls -F


20170201.tar.bz2  dataset/  jawiki/  ldcc-20140209.tar.gz  w2v/


In [ ]:
# # Word2Vecの設定をする（100秒程度かかる）
# from gensim.models import word2vec

# word2vec_model = word2vec.Word2Vec(
#         sentences, # 分かち書き，クリーニング処理，正規化処理済み単語リスト（コーパス）
#         sg=1, # 学習アルゴリズム，1ならskip-gram，0ならCBOW．
#     )

In [ ]:
# Word2Vecで学習する（1エポックで約70秒，10エポックで約10分）

from gensim.models import word2vec
import logging

# ログ取得の設定をする
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# コーパスを指定する　# 分かち書き，クリーニング処理，正規化処理済み単語リスト
# sentences
# sentences = word2vec.Text8Corpus('./copus/jawiki_corpus_wakati.txt')

# ベクターサイズ（ベクトルの次元）200，最低出現頻度5，ウィンドゥ幅15, skip-gram で学習する
epoch = 10 # 約12分かかる
model_ldn = word2vec.Word2Vec(sentences, vector_size=200, min_count=5, window=15, sg=1, epochs=epoch)


In [ ]:
# 学習済み単語ベクトルをファイルに出力する
model_ldn.wv.save_word2vec_format('./w2v/ldn_w2v.vec', binary=True)

# 単語ベクトルで加減算・類似度・距離などの演算を行う

Word2Vecの使用法は[マニュアル](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)を参照

In [ ]:
# 単語ベクトルファイルの確認

!ls -l w2v

total 20913
-rw------- 1 root root 21414888 Jun 20 22:49 ldn_w2v.vec


In [ ]:
# 単語ベクトルファイルの読み込み
from gensim.models import KeyedVectors

model_ldn_w2v =KeyedVectors.load_word2vec_format('./w2v/ldn_w2v.vec', binary=True)


In [ ]:
# 単語ベクトル（200次元）
wv1 = model_ldn_w2v['山形']
print(wv1.shape)
print(wv1)

(200,)
[-0.06388173 -0.02027029  0.35223556 -0.08359013  0.1492417  -0.25070816
 -0.1610405   0.516708    0.01149047  0.14671847 -0.29057816  0.00456569
  0.2656718   0.13024145 -0.12059033  0.22109737  0.5494895   0.5230991
  0.01493447 -0.56451803 -0.25565642  0.06269325  0.18837078 -0.13515626
  0.03873313  0.27983698  0.14213715  0.27198502 -0.06410703  0.11774141
  0.30702043  0.40249345  0.10785269 -0.38087475  0.01208199 -0.55145043
  0.1053562  -0.30517364  0.3972101  -0.31051126 -0.35509142  0.03921518
 -0.22388092 -0.02642834  0.18578939  0.11962558 -0.4210954  -0.28161094
  0.14001876  0.25655037 -0.47155133 -0.2583695  -0.38137087  0.06115125
  0.19503717 -0.10974523  0.22441883 -0.49940106 -0.0293592  -0.26985824
 -0.21183297  0.0099112  -0.05550761  0.2942753  -0.1352657   0.1617146
  0.03903295 -0.03919673  0.17759831  0.14282484  0.6389827  -0.73278016
  0.18187574  0.39208883 -0.29032052  0.57729346 -0.00624618 -0.30605978
 -0.06890649 -0.26287714  0.5255539  -0.479066

In [ ]:
# 単語の類似度
similarity = model_ldn_w2v.similarity('山形', '山形')
print(similarity)

1.0


In [ ]:
# 単語の類似度
similarity = model_ldn_w2v.similarity('山形', 'ニューヨーク')
print(similarity)

0.21549487


In [ ]:
# 単語の距離
distance = model_ldn_w2v.distance('山形', '山形')
print(distance)

0.0


In [ ]:
# 単語の距離
distance = model_ldn_w2v.distance('山形', 'ニューヨーク')
print(distance)

0.7845051288604736


In [ ]:
# 単語の演算　山形と近いのは
words = model_ldn_w2v.similar_by_word('山形', topn=10)
for word in words:
  print(word)

('盛岡', 0.5482286810874939)
('薩摩', 0.5297088623046875)
('丸亀', 0.5193943977355957)
('幕張', 0.5173351168632507)
('滋賀', 0.5172857642173767)
('府中', 0.5044071078300476)
('南国', 0.5032909512519836)
('トシオ', 0.5024840831756592)
('西宮', 0.5022903084754944)
('新潟', 0.49498793482780457)


In [ ]:
# 単語の演算　山形と川と言ったら？
words = model_ldn_w2v.most_similar(positive=['山形','川'], topn=5)
for word in words:
  print(word)

('盛岡', 0.6339346766471863)
('大岡', 0.600226879119873)
('丸亀', 0.5940335392951965)
('吉祥寺', 0.5924842953681946)
('薩摩', 0.5825843811035156)


In [ ]:
# 類似の単語
words = model_ldn_w2v.most_similar(positive=['フルート', 'ピアノ'], topn=3)
for word in words:
  print(word)

KeyError: "Key 'フルート' not present in vocabulary"

In [ ]:
# 単語の演算　王様 - 男 + 女　は？（cos距離）
words = model_ldn_w2v.most_similar_cosmul(positive=['王様', '女'], negative=['男'], topn=1)
for word in words:
  print(word)

('涼', 0.8140841722488403)


In [ ]:
# 単語の演算　ロンドン - イギリス + 日本　は？
words = model_ldn_w2v.most_similar(positive=['ロンドン','日本'], negative=['イギリス'], topn=3)
for word in words:
  print(word)

('五輪', 0.6290484070777893)
('出場', 0.5611072778701782)
('代表', 0.548005223274231)


In [ ]:
# 仲間外れの単語は？

l = ["金沢", "ピアノ", "盛岡", "仙台", "水戸", "甲府"]
words = model_ldn_w2v.doesnt_match(l)
print(words)
# for word in words:
#   print(word)

ピアノ


In [ ]:
# 仲間外れの単語は？

l = ["金沢", "福岡", "盛岡", "仙台", "水戸", "甲府"]
words = model_ldn_w2v.doesnt_match(l)
print(words)

福岡


In [ ]:
# 単語の演算　猫といったら？
words = model_ldn_w2v.similar_by_word('猫', topn=3)
for word in words:
  print(word)

('ひろし', 0.6829724907875061)
('カンボジア', 0.5780990719795227)
('中平', 0.5157045722007751)


In [ ]:
# 単語の演算　ねこといったら？
words = model_ldn_w2v.similar_by_word('ねこ', topn=3)
for word in words:
  print(word)

('鳴き声', 0.6629879474639893)
('ぃっしんぐ', 0.6547021865844727)
('忍者', 0.6520165801048279)


# まとめ
- 単語ベクトルを使えば単語間の演算ができることが理解できた
- 日本語自然言語処理の基本手順が理解できた
- 日本語自然言語処理で使われる用語が理解できた
- 日本語自然言語処理の代表的なツールの使い方が理解できた
- ニューラルネットワークを用いたWord2Vecの学習の仕組みが理解できた
- Word2Vecで単語間の関係を反映した単語ベクトルを作成することができた
- コーパスの種類やコーパスの大きさで単語ベクトル値が変わり，「賢さ」にも影響することが分かった